In [ ]:
def gen_contours_new(data, center, R_electrode, r_um, downsampling = 15.38, num_contours=8):
    """
    Generate a mask image with circular equi-distance contours centered to predefined electrode center
    data: histology images zStacked. x_pxl x y_pxl x img_planes
    center: predefined center point for the electrodes, [x, y]
    r_um: radius of contours in micrometers
    R_electrode: diameter of electrode cross section
    """
    r=np.round(r_um/0.325).astype(int)
    r_base_px = np.round(R_electrode/0.325/2).astype(int)
    cx,cy = np.round(center*downsampling).astype(int)
    mask=np.ones_like(data[:,:,0])
    # num_regions=int(limit_um/r_um)
    
    if r_base_px:
        roi=skimage.morphology.disk(r_base_px)        
        mask[cx-r_base_px:cx+r_base_px+1, cy-r_base_px:cy+r_base_px+1]=mask[cx-r_base_px:cx+r_base_px+1, cy-r_base_px:cy+r_base_px+1]+roi
            
    for i in range(1,num_contours+1):
        rTmp = r*i + r_base_px
        roi=skimage.morphology.disk(rTmp)        
        mask[cx-rTmp:cx+rTmp+1, cy-rTmp:cy+rTmp+1]=mask[cx-rTmp:cx+rTmp+1, cy-rTmp:cy+rTmp+1]+roi

    if r_base:
        mask[mask==num_regions+2]=0
    # else:
    #     mask=i-mask+1
        
    return num_regions, mask

In [ ]:
def mask_vessels(mask, v_x, v_y, v_r):
    roi=skimage.morphology.disk(v_r)*999
    mask[v_x-v_r:v_x+v_r+1, v_y-v_r:v_y+v_r+1]=mask[v_x-v_r:v_x+v_r+1, v_y-v_r:v_y+v_r+1]+roi
    # mask=mask*(mask>0)
    return mask


def mask_stitches(mask):
    mask[1925:2050, :] = 0
    mask[3850:3950, :] = 0
    mask[:, 1900:2000] = 0
    mask[:, 3800:3900] = 0
    return mask


def set_reference(mask):
    """
    Sets a reference area at the surrounding edges of the image
    """
    width = 1500
    tmpMask = mask
    onesMask = (mask == 1).astype(int)
    
    onesMask[width:-width,width:-width] = onesMask[width:-width,width:-width]* -1
    onesMask[onesMask==1] = 0
    mask = mask + onesMask 
    
    return mask

In [ ]:
def gen_contours(data, center, r_base, r_um, limit_um=None):
    """
    Generate a mask image with circular equi-distance contours centered to predefined electrode center
    data: histology images zStacked. x_pxl x y_pxl x img_planes
    center: predefined center point for the electrodes, [x, y]
    r: radius of contours in micrometers
    """
    r=np.round(r_um/0.325).astype(int)
    # r_base_px = np.round(r_base/0.325).astype(int)
    r_base_px = np.round(r_base/2).astype(int)
    cx,cy = center
    #mask=np.ones_like(data[:,:,0])
    mask=np.ones_like(data)
    num_regions=int(limit_um/r_um)
    
    if r_base_px:
        roi=skimage.morphology.disk(r_base_px)        
        mask[cx-r_base_px:cx+r_base_px+1, cy-r_base_px:cy+r_base_px+1]=mask[cx-r_base_px:cx+r_base_px+1, cy-r_base_px:cy+r_base_px+1]+roi
            
    for i in range(1,num_regions+1):
        rTmp = r*i + r_base_px
        roi=skimage.morphology.disk(rTmp)        
        mask[cx-rTmp:cx+rTmp+1, cy-rTmp:cy+rTmp+1]=mask[cx-rTmp:cx+rTmp+1, cy-rTmp:cy+rTmp+1]+roi

    if r_base:
        mask[mask==num_regions+2]=0
    # else:
    #     mask=i-mask+1
        
    return num_regions, mask


def gen_control_area(mask, r_base, r_um, limit_um, control_location, offset=0):
    """
    Generates a control area with contours within the image. IF NO control image exists separately. 
    """
    r=np.round(r_um/0.325).astype(int)
    r_base_px = np.round(r_base/0.325).astype(int)
    limit=np.round(limit_um/0.325).astype(int)
    offset = np.round(offset/0.325).astype(int)
    num_regions=int(limit_um/r_um)

    if offset:
        limit=offset
    
    if control_location==1:
        cx=limit+r_base_px+2 
        cy=limit+r_base_px+2 
        
    elif control_location==2:
        cx=limit+r_base_px+2 
        cy=np.shape(mask)[1]-limit-r_base_px-2 

    elif control_location==3:
        cx=np.shape(mask)[0]-limit-r_base_px-2 
        cy=limit+r_base_px+2 
        
    elif control_location==4:
        cx=np.shape(mask)[0]-limit-r_base_px-2 
        cy=np.shape(mask)[1]-limit-r_base_px-2

        # cx=limit+r_base_px+2 
    # cy=limit+r_base_px+2 
    # if r_base:
    #     roi=skimage.morphology.disk(r_base_px)*10       
    #     mask[cx-r_base_px:cx+r_base_px+1, cy-r_base_px:cy+r_base_px+1]=mask[cx-r_base_px:cx+r_base_px+1, cy-r_base_px:cy+r_base_px+1]+roi            
    # for i in range(1,num_regions+1):
    #     rTmp = r*i + r_base_px
    #     roi=skimage.morphology.disk(rTmp)*10        
    #     mask[cx-rTmp:cx+rTmp+1, cy-rTmp:cy+rTmp+1]=mask[cx-rTmp:cx+rTmp+1, cy-rTmp:cy+rTmp+1]+roi

    # cx=limit+r_base_px+2 
    # cy=np.shape(mask)[1]-limit-r_base_px-2 
    # if r_base:
    #     roi=skimage.morphology.disk(r_base_px)*10       
    #     mask[cx-r_base_px:cx+r_base_px+1, cy-r_base_px:cy+r_base_px+1]=mask[cx-r_base_px:cx+r_base_px+1, cy-r_base_px:cy+r_base_px+1]+roi       
    # for i in range(1,num_regions+1):
    #     rTmp = r*i + r_base_px
    #     roi=skimage.morphology.disk(rTmp)*10        
    #     mask[cx-rTmp:cx+rTmp+1, cy-rTmp:cy+rTmp+1]=mask[cx-rTmp:cx+rTmp+1, cy-rTmp:cy+rTmp+1]+roi

    # cx=np.shape(mask)[0]-limit-r_base_px-2 
    # cy=limit+r_base_px+2
    # if r_base:
    #     roi=skimage.morphology.disk(r_base_px)*10       
    #     mask[cx-r_base_px:cx+r_base_px+1, cy-r_base_px:cy+r_base_px+1]=mask[cx-r_base_px:cx+r_base_px+1, cy-r_base_px:cy+r_base_px+1]+roi       
    # for i in range(1,num_regions+1):
    #     rTmp = r*i + r_base_px
    #     roi=skimage.morphology.disk(rTmp)*10        
    #     mask[cx-rTmp:cx+rTmp+1, cy-rTmp:cy+rTmp+1]=mask[cx-rTmp:cx+rTmp+1, cy-rTmp:cy+rTmp+1]+roi

    # cx=np.shape(mask)[0]-limit-r_base_px-2 
    # cy=np.shape(mask)[1]-limit-r_base_px-2
    if r_base:
        roi=skimage.morphology.disk(r_base_px)*10       
        mask[cx-r_base_px:cx+r_base_px+1, cy-r_base_px:cy+r_base_px+1]=mask[cx-r_base_px:cx+r_base_px+1, cy-r_base_px:cy+r_base_px+1]+roi       
    for i in range(1,num_regions+1):
        rTmp = r*i + r_base_px
        roi=skimage.morphology.disk(rTmp)*10        
        mask[cx-rTmp:cx+rTmp+1, cy-rTmp:cy+rTmp+1]=mask[cx-rTmp:cx+rTmp+1, cy-rTmp:cy+rTmp+1]+roi

    
    mask[mask==10*(num_regions+1)+1]=0
    
    return mask